# Census / Chicago Block Party Analysis
The hypothesis is that the number of block parties in a given area is correlated with the home ownership in that area. 
This script will analyze the data to determine if this is true.

In [1]:
# Read in api keys
with open('api_keys.txt', 'r') as file:
    file_keys = file.readlines()
file_keys
key = str(file_keys[0]).strip()

In [2]:
# Chicago Zip Codes
################################################################################
zip_codes = [
    '60647', '60639', '60707', '60622', '60651', '60611', '60638', '60652',
    '60626', '60615', '60621', '60645', '60643', '60660', '60640', '60614',
    '60631', '60646', '60628', '60625', '60641', '60657', '60636', '60649',
    '60617', '60633', '60612', '60604', '60624', '60656', '60644', '60655',
    '60603', '60605', '60653', '60609', '60666', '60618', '60616', '60602',
    '60601', '60608', '60607', '60661', '60606', '60827', '60630', '60642',
    '60659', '60634', '60613', '60610', '60654', '60632', '60623', '60629',
    '60620', '60637', '60619'
    ]

In [18]:
################################################################################
# Define Variables Used In API Request
################################################################################

# Set variables for API endpoint and key
endpoint = "https://api.census.gov/data"

# The API limits the number of variables to 50 at a time
MAX_CODE_CHUNK = 49

years = [2022]

variables = [
    # ACS Profile Data
    {
        'dataset':'acs/acs5/profile',
        'time':years,
        'codes':{
            'Total households estimate':'DP02_0001E',
            'Total households MOE':'DP02_0001M'
        }
    },
    # ACS Detail Data
    {
        'dataset':'acs/acs5',
        'time':years,
        'codes':{
            'Estimate!!Total:Pop':'B01001_001E',
            'Estimate!!Total:Pop:MOE':'B01001_001M',
            'Estimate!!Total:':'B25003_001E',
            'Estimate!!Total:MOE':'B25003_001M',
            'Estimate!!Total:!!Owner occupied:':'B25003_002E', ### Owner Occupied
            'Estimate!!Total:!!Owner occupied:MOE':'B25003_002M',
            'Estimate!!Total:!!Renter occupied:':'B25003_003E', ### Renter Occupied
            'Estimate!!Total:!!Renter occupied:MOE':'B25003_003M',
            'Estimate!!Total_:':'B25032_001E',  ### Total Units
            'Estimate!!Total_:MOE':'B25032_001M',
            'Estimate!!Total_Tenure':'B25038_001E',
            'Estimate!!Total:!!Owner occupied:Tenure':'B25038_002E',
            'Estimate!!Total:!!Owner occupied:!!Moved in 2021 or later':'B25038_003E',
            'Estimate!!Total:!!Owner occupied:!!Moved in 2018 to 2020':'B25038_004E',
            'Estimate!!Total:!!Owner occupied:!!Moved in 2010 to 2017':'B25038_005E',
            'Estimate!!Total:!!Owner occupied:!!Moved in 2000 to 2009':'B25038_006E',
            'Estimate!!Total:!!Owner occupied:!!Moved in 1990 to 1999':'B25038_007E',
            'Estimate!!Total:!!Owner occupied:!!Moved in 1989 or earlier':'B25038_008E',
        }
    }
]

## Load Census Data

In [19]:
# Load necessary libraries
import requests
import time
import pandas as pd
from urllib.parse import quote


################################################################################
# Loop Through Variables & Make API Call & Collect Results
################################################################################
time_periods = []
for vars in variables:
    time_period = []
    for year in vars['time']:
        chunk_results = pd.DataFrame(columns=['NAME'])
        for i in range(0, len(vars['codes'].values()), MAX_CODE_CHUNK):
            codes = list(vars['codes'].values())
            codes_subset = ['NAME'] + codes[i:i + MAX_CODE_CHUNK]
            print(codes[i:i + MAX_CODE_CHUNK])
        
            # Construct API request URL
            location = "zip code tabulation area:" + ','.join(zip_codes)
            url_vars = ','.join(codes_subset)
            dataset = vars['dataset']
            url = f"{endpoint}/{year}/{dataset}?get={url_vars}&for={location}&key={key}"
            encoded_url = quote(url, safe=':/?=&')
            
            # Send API request and convert response to data frame
            print(encoded_url)
            response = requests.get(encoded_url)
            df_response = pd.DataFrame(response.json()[1:], columns=response.json()[0])
            chunk_results = pd.merge(chunk_results, df_response, how='outer').copy()
            
            # Sleep for 1/4 sec to avoid rate limits
            time.sleep(0.25)
        
        # Add the results of each year to list
        chunk_results.insert(0, 'Year', year)
        time_period.append(chunk_results)

    # Combine all of the data into one Pandas Table
    time_period = pd.concat(time_period, axis=1)
    time_periods.append(time_period)
    
census_data_df = pd.concat(time_periods, axis=1)

['DP02_0001E', 'DP02_0001M']
https://api.census.gov/data/2022/acs/acs5/profile?get=NAME%2CDP02_0001E%2CDP02_0001M&for=zip%20code%20tabulation%20area:60647%2C60639%2C60707%2C60622%2C60651%2C60611%2C60638%2C60652%2C60626%2C60615%2C60621%2C60645%2C60643%2C60660%2C60640%2C60614%2C60631%2C60646%2C60628%2C60625%2C60641%2C60657%2C60636%2C60649%2C60617%2C60633%2C60612%2C60604%2C60624%2C60656%2C60644%2C60655%2C60603%2C60605%2C60653%2C60609%2C60666%2C60618%2C60616%2C60602%2C60601%2C60608%2C60607%2C60661%2C60606%2C60827%2C60630%2C60642%2C60659%2C60634%2C60613%2C60610%2C60654%2C60632%2C60623%2C60629%2C60620%2C60637%2C60619&key=e4ba254c7ccb616296fa1ecf9f0cf07d4df55ab2
['B01001_001E', 'B01001_001M', 'B25003_001E', 'B25003_001M', 'B25003_002E', 'B25003_002M', 'B25003_003E', 'B25003_003M', 'B25032_001E', 'B25032_001M', 'B25038_001E', 'B25038_002E', 'B25038_003E', 'B25038_004E', 'B25038_005E', 'B25038_006E', 'B25038_007E', 'B25038_008E']
https://api.census.gov/data/2022/acs/acs5?get=NAME%2CB01001_001E%

In [20]:
census_data_df

,Year,NAME,DP02_0001E,DP02_0001M,zip code tabulation area,Year,NAME,B01001_001E,B01001_001M,B25003_001E,...,B25032_001M,B25038_001E,B25038_002E,B25038_003E,B25038_004E,B25038_005E,B25038_006E,B25038_007E,B25038_008E,zip code tabulation area
0,2022,ZCTA5 60601,9252,767,60601,2022,ZCTA5 60601,16398,1721,9252,...,767,9252,2820,31,447,942,822,420,158,60601
1,2022,ZCTA5 60602,468,231,60602,2022,ZCTA5 60602,938,495,468,...,231,468,267,20,26,196,25,0,0,60602
2,2022,ZCTA5 60603,674,244,60603,2022,ZCTA5 60603,1110,319,674,...,244,674,458,13,104,304,28,9,0,60603
3,2022,ZCTA5 60604,452,112,60604,2022,ZCTA5 60604,737,197,452,...,112,452,170,0,13,58,99,0,0,60604
4,2022,ZCTA5 60605,18817,1192,60605,2022,ZCTA5 60605,34314,2076,18817,...,1192,18817,8035,114,1448,3577,2053,704,139,60605
5,2022,ZCTA5 60606,2666,617,60606,2022,ZCTA5 60606,3513,704,2666,...,617,2666,879,114,130,272,257,106,0,60606
6,2022,ZCTA5 60607,13474,774,60607,2022,ZCTA5 60607,28797,1684,13474,...,774,13474,6179,428,786,2992,1185,441,347,60607
7,2022,ZCTA5 60608,29081,1025,60608,2022,ZCTA5 60608,82749,3174,29081,...,1025,29081,11492,243,1497,2983,2796,1652,2321,60608
8,2022,ZCTA5 60609,22973,1056,60609,2022,ZCTA5 60609,63241,3238,22973,...,1056,22973,9103,160,835,2582,2355,1224,1947,60609
9,2022,ZCTA5 60610,25622,1193,60610,2022,ZCTA5 60610,42904,2069,25622,...,1193,25622,9607,365,1437,3616,2263,1005,921,60610


In [165]:
# create empty data frame to hold results
summary_df = census_data_df.iloc[:, 0:2].copy()

# Add the number of households
summary_df['n_households'] = census_data_df.loc[:, 'DP02_0001E'].astype('int32')

# Calculate the percentage of households that are owner occupied
owner_totals = census_data_df['B25003_001E'].astype('int32')
summary_df['h_own_pct'] = census_data_df.loc[:, 'B25003_002E'].astype('int32')\
    .div(owner_totals, axis=0)

# Calculate the percentage of home ownership by tenure
home_owner_tenure_totals = census_data_df['B25038_002E'].astype('int32')
summary_df['h_own_tenure_0-1'] = census_data_df.loc[:, 'B25038_003E'].astype('int32')\
    .div(home_owner_tenure_totals, axis=0)
summary_df['h_own_tenure_2-4'] = census_data_df.loc[:, 'B25038_004E'].astype('int32')\
    .div(home_owner_tenure_totals, axis=0)
summary_df['h_own_tenure_5-12'] = census_data_df.loc[:, 'B25038_005E'].astype('int32')\
    .div(home_owner_tenure_totals, axis=0)
summary_df['h_own_tenure_13-22'] = census_data_df.loc[:, 'B25038_006E'].astype('int32')\
    .div(home_owner_tenure_totals, axis=0)
summary_df['h_own_tenure_23-32'] = census_data_df.loc[:, 'B25038_007E'].astype('int32')\
    .div(home_owner_tenure_totals, axis=0)
summary_df['h_own_tenure_33+'] = census_data_df.loc[:, 'B25038_008E'].astype('int32')\
    .div(home_owner_tenure_totals, axis=0)

cols = ['B25038_003E','B25038_004E','B25038_005E','B25038_006E','B25038_007E','B25038_008E']
wts = list(range(1, len(cols)+1))
summary_df['tenure_idx'] = census_data_df.loc[:, cols].astype('int32')\
    .multiply(wts, axis=1)\
    .sum(axis=1)\
    .divide(census_data_df['B25038_002E'].astype('int32'), axis=0)

# home_owner_rate
summary_df['NAME'] = [int(str(s)[5:]) for s in summary_df['NAME']]
summary_df\
    .query("NAME==60643 | NAME==60620 | NAME==60652")

,Year,NAME,n_households,h_own_pct,h_own_tenure_0-1,h_own_tenure_2-4,h_own_tenure_5-12,h_own_tenure_13-22,h_own_tenure_23-32,h_own_tenure_33+,tenure_idx
19,2022,60620,26150,0.485545,0.031897,0.080491,0.163503,0.187682,0.120343,0.416083,4.532330
40,2022,60643,18197,0.743584,0.015446,0.079004,0.193482,0.187717,0.167246,0.357106,4.483630
47,2022,60652,13643,0.849153,0.029089,0.050755,0.238757,0.271385,0.283297,0.126716,4.109193


## Adding City Block Club Permit Data
Data Link: [Data Portal](https://data.cityofchicago.org/Transportation/Transportation-Department-Permits/pubx-yq2d/explore/query/SELECT%0A%20%20%60uniquekey%60%2C%0A%20%20%60applicationnumber%60%2C%0A%20%20%60applicationtype%60%2C%0A%20%20%60applicationdescription%60%2C%0A%20%20%60worktype%60%2C%0A%20%20%60worktypedescription%60%2C%0A%20%20%60applicationstatus%60%2C%0A%20%20%60currentmilestone%60%2C%0A%20%20%60applicationstartdate%60%2C%0A%20%20%60applicationenddate%60%2C%0A%20%20%60applicationprocesseddate%60%2C%0A%20%20%60applicationissueddate%60%2C%0A%20%20%60applicationfinalizeddate%60%2C%0A%20%20%60applicationexpiredate%60%2C%0A%20%20%60applicationname%60%2C%0A%20%20%60comments%60%2C%0A%20%20%60totalfees%60%2C%0A%20%20%60waivedfees%60%2C%0A%20%20%60primarycontactlast%60%2C%0A%20%20%60primarycontactfirst%60%2C%0A%20%20%60primarycontactmiddle%60%2C%0A%20%20%60primarycontactstreet%60%2C%0A%20%20%60primarycontactstreet2%60%2C%0A%20%20%60primarycontactcity%60%2C%0A%20%20%60primarycontactstate%60%2C%0A%20%20%60primarycontactzip%60%2C%0A%20%20%60emergencycontactname%60%2C%0A%20%20%60lastinspectionnumber%60%2C%0A%20%20%60lastinspectiontype%60%2C%0A%20%20%60lastinsptypedescr%60%2C%0A%20%20%60lastinspectiondate%60%2C%0A%20%20%60lastinspectionresult%60%2C%0A%20%20%60streetnumberfrom%60%2C%0A%20%20%60streetnumberto%60%2C%0A%20%20%60direction%60%2C%0A%20%20%60streetname%60%2C%0A%20%20%60suffix%60%2C%0A%20%20%60placement%60%2C%0A%20%20%60streetclosure%60%2C%0A%20%20%60detail%60%2C%0A%20%20%60parkingmeterpostingorbagging%60%2C%0A%20%20%60ward%60%2C%0A%20%20%60xcoordinate%60%2C%0A%20%20%60ycoordinate%60%2C%0A%20%20%60latitude%60%2C%0A%20%20%60longitude%60%2C%0A%20%20%60location%60%0AWHERE%0A%20%20caseless_one_of%28%60worktype%60%2C%20%22BlockParty%22%29%0A%20%20AND%20caseless_one_of%28%60currentmilestone%60%2C%20%22Complete%22%29/page/filter)

In [167]:
df_permits = pd.read_csv('BlockPartyData.csv')
df_permits['Year'] = pd.to_datetime(df_permits['APPLICATIONFINALIZEDDATE']).dt.year
df_permits = df_permits[df_permits['Year'] == 2022]
df_permits['PRIMARYCONTACTZIP'] = [z[:5] for z in df_permits['PRIMARYCONTACTZIP'].astype('str')]

# filter rows where zip code is not a number
df_permits = df_permits[df_permits['PRIMARYCONTACTZIP'].apply(lambda x: str(x).isdigit() and len(str(x)[:5]) == 5)]
print(df_permits.shape) # print the shape of the data frame

# Compute Zip Code Summary Statistic
df_permits['PRIMARYCONTACTZIP'] = df_permits['PRIMARYCONTACTZIP'].astype('int')
zip_summary = df_permits.groupby(['PRIMARYCONTACTZIP']).agg({'APPLICATIONNUMBER':'count'}).reset_index()
zip_summary.columns = ['ZipCode', 'n_permits']
zip_summary.head(10)

(3657, 48)


C:\Users\aaren\AppData\Local\Temp\ipykernel_24748\1205965248.py:1: DtypeWarning: Columns (28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df_permits = pd.read_csv('BlockPartyData.csv')


,ZipCode,n_permits
0,60015,1
1,60104,1
2,60301,3
3,60303,1
4,60402,1
5,60411,1
6,60475,4
7,60559,1
8,60602,4
9,60603,1


## Merge Census and City Data

In [169]:
df_merged = pd.merge(summary_df, zip_summary, left_on='NAME', right_on='ZipCode', how='left')\
    .drop(columns=['ZipCode'])\
    .assign(permit_rate=lambda x: x['n_permits'].fillna(0)/(x['n_households']/100) )\
    .sort_values('permit_rate', ascending=False)
print(f"Correlation between permit rate and home ownership rate: {df_merged['permit_rate'].corr(df_merged['h_own_pct'])}")
print(f"Correlation between permit rate and home ownership tenure: {df_merged['permit_rate'].corr(df_merged['tenure_idx'])}")
df_merged

Correlation between permit rate and home ownership rate: 0.6010380927967995
Correlation between permit rate and home ownership tenure: 0.40273878618852765


,Year,NAME,n_households,h_own_pct,h_own_tenure_0-1,h_own_tenure_2-4,h_own_tenure_5-12,h_own_tenure_13-22,h_own_tenure_23-32,h_own_tenure_33+,tenure_idx,n_permits,permit_rate
50,2022,60655,10672,0.845202,0.018847,0.074058,0.200887,0.243016,0.193792,0.269401,4.327051,201.0,1.883433
29,2022,60631,11977,0.743258,0.026848,0.093013,0.210851,0.253089,0.182319,0.233880,4.172658,130.0,1.085414
28,2022,60630,21340,0.675820,0.021634,0.103245,0.303980,0.189294,0.174040,0.207808,4.014284,188.0,0.880975
1,2022,60602,468,0.570513,0.074906,0.097378,0.734082,0.093633,0.000000,0.000000,2.846442,4.0,0.854701
40,2022,60643,18197,0.743584,0.015446,0.079004,0.193482,0.187717,0.167246,0.357106,4.483630,144.0,0.791339
23,2022,60624,13358,0.278859,0.003758,0.093691,0.193557,0.211275,0.153020,0.344698,4.450201,101.0,0.756101
43,2022,60646,10806,0.805664,0.010567,0.137377,0.222261,0.190214,0.234436,0.205146,4.116012,81.0,0.749584
47,2022,60652,13643,0.849153,0.029089,0.050755,0.238757,0.271385,0.283297,0.126716,4.109193,91.0,0.667009
20,2022,60621,11134,0.253907,0.000000,0.037496,0.188893,0.238769,0.145030,0.389813,4.660771,71.0,0.637686
17,2022,60618,37060,0.505019,0.034676,0.178564,0.360387,0.169908,0.150085,0.106380,3.541302,231.0,0.623314


#### Summary:
It appears that home ownership is moderately/strongly correlated with the number of block party permits scaled by the number of households around 60%. The length of home ownership is not as strongly correlated at 40%.

Other notes:
- The block party data is based on only completed block party permits in Chicago with a start date in 2022. 